# Text Summarization

In [5]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
from nltk.tokenize import sent_tokenize
import numpy as np
import networkx as nx
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ezifakhrezi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Load Dataset

In [6]:
def read_article(file_name):
    """
    Generate clean sentences from the file
    Input : path file name
    Output : array of sentences
    """
    
    file = open(file_name, "r")
    filedata = file.readlines()
    sentences = []   
    for sentence in filedata:
        print("Sentence : {}".format(sentence))
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" ")) # filter charachter only
    
    return sentences

In [7]:
def read_article_2(filename):
    """
    Generate clean sentences from the, split it to sentences
    Input : path file name
    Output : array of sentences
    """
    file = open(filename, "r")
    filedata = file.readlines()
    sentences = sent_tokenize(filedata[0])
    return sentences

In [8]:
#sentences = [read_article_2(path+file_name) for file_name in file_names]
#sentences

### Samples sentences

In [9]:
path = "data set/data tafsir/"
file_names = ['19-85-10.txt',
 '19-85-2.txt',
 '19-85-9.txt',
 '19-85-5.txt',
 '19-85-1.txt',
 '19-85-8.txt',
 '19-85-4.txt',
 '19-85-7.txt',
 '19-85-3.txt',
 '19-85-6.txt']

sentences = [read_article(path+file_name) for file_name in file_names]
sentences

Sentence : "Dan sembahan-sembahan yang mereka sembah selain Allah tidak dapat memberi syafaat; akan tetapi orang yang dapat memberi syafaat ialah yang mengakui yang Haq, yakni keesaan Allah dan mereka meyakininya" (QS. az-Zukhruf [43]: 86). Demikian juga firman-Nya: orang "Tiadalah berguna syafaat di sisi Allah melainkan bagi orang yang telah diizinkan-Nya memeroleh syafaat itu sehingga apabila telah dihilangkan ketakutan dari hati mereka, mereka berkata: 'Apakah yang telah difirmankan oleh Tuhan-mu?’ Mereka menjawab: Putusan yang benar', dan Dia-lah Yang Mahatinggi lagi Mahabesar" (QS. Saba' [34]: 23). Memang, syafaat ala dunia tidak akan terjadi di hari Kemudian, dan itulah yang dinafikan oleh sekian ayat. 
Sentence : Jika apa yang diharapkan seseorang terdapat pada pihak lain, yang ditakuti atau disegani, dia dapat menuju kepadanya dengan menggenapkan dirinya dengan orang lain itu untuk bersama-sama memohon kepada yang ditakuti dan disegani itu. Orang lain itulah yang mengajukan per

[[['"Dan',
   'sembahan-sembahan',
   'yang',
   'mereka',
   'sembah',
   'selain',
   'Allah',
   'tidak',
   'dapat',
   'memberi',
   'syafaat;',
   'akan',
   'tetapi',
   'orang',
   'yang',
   'dapat',
   'memberi',
   'syafaat',
   'ialah',
   'yang',
   'mengakui',
   'yang',
   'Haq,',
   'yakni',
   'keesaan',
   'Allah',
   'dan',
   'mereka',
   'meyakininya"',
   '(QS.',
   'az-Zukhruf',
   '[43]:',
   '86).',
   'Demikian',
   'juga',
   'firman-Nya:',
   'orang',
   '"Tiadalah',
   'berguna',
   'syafaat',
   'di',
   'sisi',
   'Allah',
   'melainkan',
   'bagi',
   'orang',
   'yang',
   'telah',
   'diizinkan-Nya',
   'memeroleh',
   'syafaat',
   'itu',
   'sehingga',
   'apabila',
   'telah',
   'dihilangkan',
   'ketakutan',
   'dari',
   'hati',
   'mereka,',
   'mereka',
   'berkata:',
   "'Apakah",
   'yang',
   'telah',
   'difirmankan',
   'oleh',
   'Tuhan-mu?’',
   'Mereka',
   'menjawab:',
   'Putusan',
   'yang',
   "benar',",
   'dan',
   'Dia-lah',
   '

In [10]:
len(sentences)

10

In [11]:
sentences[0]

[['"Dan',
  'sembahan-sembahan',
  'yang',
  'mereka',
  'sembah',
  'selain',
  'Allah',
  'tidak',
  'dapat',
  'memberi',
  'syafaat;',
  'akan',
  'tetapi',
  'orang',
  'yang',
  'dapat',
  'memberi',
  'syafaat',
  'ialah',
  'yang',
  'mengakui',
  'yang',
  'Haq,',
  'yakni',
  'keesaan',
  'Allah',
  'dan',
  'mereka',
  'meyakininya"',
  '(QS.',
  'az-Zukhruf',
  '[43]:',
  '86).',
  'Demikian',
  'juga',
  'firman-Nya:',
  'orang',
  '"Tiadalah',
  'berguna',
  'syafaat',
  'di',
  'sisi',
  'Allah',
  'melainkan',
  'bagi',
  'orang',
  'yang',
  'telah',
  'diizinkan-Nya',
  'memeroleh',
  'syafaat',
  'itu',
  'sehingga',
  'apabila',
  'telah',
  'dihilangkan',
  'ketakutan',
  'dari',
  'hati',
  'mereka,',
  'mereka',
  'berkata:',
  "'Apakah",
  'yang',
  'telah',
  'difirmankan',
  'oleh',
  'Tuhan-mu?’',
  'Mereka',
  'menjawab:',
  'Putusan',
  'yang',
  "benar',",
  'dan',
  'Dia-lah',
  'Yang',
  'Mahatinggi',
  'lagi',
  'Mahabesar"',
  '(QS.',
  "Saba'",
  '[34

## Merge Sentences

In [12]:
def merge_sentences(sentences):
    """
    Merge sentences to one array
    """
    full_sentences = []
    for sentence in sentences:
        for arr_word in sentence:
            full_sentences.append(arr_word)
    return full_sentences

In [13]:
full_sentences = merge_sentences(sentences)
full_sentences


[['"Dan',
  'sembahan-sembahan',
  'yang',
  'mereka',
  'sembah',
  'selain',
  'Allah',
  'tidak',
  'dapat',
  'memberi',
  'syafaat;',
  'akan',
  'tetapi',
  'orang',
  'yang',
  'dapat',
  'memberi',
  'syafaat',
  'ialah',
  'yang',
  'mengakui',
  'yang',
  'Haq,',
  'yakni',
  'keesaan',
  'Allah',
  'dan',
  'mereka',
  'meyakininya"',
  '(QS.',
  'az-Zukhruf',
  '[43]:',
  '86).',
  'Demikian',
  'juga',
  'firman-Nya:',
  'orang',
  '"Tiadalah',
  'berguna',
  'syafaat',
  'di',
  'sisi',
  'Allah',
  'melainkan',
  'bagi',
  'orang',
  'yang',
  'telah',
  'diizinkan-Nya',
  'memeroleh',
  'syafaat',
  'itu',
  'sehingga',
  'apabila',
  'telah',
  'dihilangkan',
  'ketakutan',
  'dari',
  'hati',
  'mereka,',
  'mereka',
  'berkata:',
  "'Apakah",
  'yang',
  'telah',
  'difirmankan',
  'oleh',
  'Tuhan-mu?’',
  'Mereka',
  'menjawab:',
  'Putusan',
  'yang',
  "benar',",
  'dan',
  'Dia-lah',
  'Yang',
  'Mahatinggi',
  'lagi',
  'Mahabesar"',
  '(QS.',
  "Saba'",
  '[34

In [14]:
full_sentences

[['"Dan',
  'sembahan-sembahan',
  'yang',
  'mereka',
  'sembah',
  'selain',
  'Allah',
  'tidak',
  'dapat',
  'memberi',
  'syafaat;',
  'akan',
  'tetapi',
  'orang',
  'yang',
  'dapat',
  'memberi',
  'syafaat',
  'ialah',
  'yang',
  'mengakui',
  'yang',
  'Haq,',
  'yakni',
  'keesaan',
  'Allah',
  'dan',
  'mereka',
  'meyakininya"',
  '(QS.',
  'az-Zukhruf',
  '[43]:',
  '86).',
  'Demikian',
  'juga',
  'firman-Nya:',
  'orang',
  '"Tiadalah',
  'berguna',
  'syafaat',
  'di',
  'sisi',
  'Allah',
  'melainkan',
  'bagi',
  'orang',
  'yang',
  'telah',
  'diizinkan-Nya',
  'memeroleh',
  'syafaat',
  'itu',
  'sehingga',
  'apabila',
  'telah',
  'dihilangkan',
  'ketakutan',
  'dari',
  'hati',
  'mereka,',
  'mereka',
  'berkata:',
  "'Apakah",
  'yang',
  'telah',
  'difirmankan',
  'oleh',
  'Tuhan-mu?’',
  'Mereka',
  'menjawab:',
  'Putusan',
  'yang',
  "benar',",
  'dan',
  'Dia-lah',
  'Yang',
  'Mahatinggi',
  'lagi',
  'Mahabesar"',
  '(QS.',
  "Saba'",
  '[34

## Build Similarity Matrix

In [15]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    
    return similarity_matrix

### Samples similarity matrix 

In [16]:
sentence_similarity_matrix = build_similarity_matrix(full_sentences,stop_words)
sentence_similarity_matrix

array([[0.        , 0.12677314, 0.05423261, 0.30697031, 0.24397502,
        0.        , 0.22821773, 0.2766993 , 0.14142136, 0.3       ],
       [0.12677314, 0.        , 0.04583492, 0.07412493, 0.20619652,
        0.08451543, 0.        , 0.13363062, 0.        , 0.16903085],
       [0.05423261, 0.04583492, 0.        , 0.0951303 , 0.        ,
        0.21693046, 0.09901475, 0.        , 0.        , 0.16269784],
       [0.30697031, 0.07412493, 0.0951303 , 0.        , 0.0855921 ,
        0.        , 0.16012815, 0.13867505, 0.12403473, 0.0877058 ],
       [0.24397502, 0.20619652, 0.        , 0.0855921 , 0.        ,
        0.        , 0.08908708, 0.23145502, 0.06900656, 0.24397502],
       [0.        , 0.08451543, 0.21693046, 0.        , 0.        ,
        0.        , 0.        , 0.15811388, 0.        , 0.1       ],
       [0.22821773, 0.        , 0.09901475, 0.16012815, 0.08908708,
        0.        , 0.        , 0.14433757, 0.12909944, 0.18257419],
       [0.2766993 , 0.13363062, 0.       

## Page Rank Sentences

In [17]:
sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
scores = nx.pagerank(sentence_similarity_graph)
scores

{0: 0.14436361369548853,
 1: 0.07963688309062442,
 2: 0.0694589216860643,
 3: 0.09778347965141448,
 4: 0.10390098902271228,
 5: 0.05988335734216622,
 6: 0.09445367999742414,
 7: 0.13834469263918103,
 8: 0.06368649539801495,
 9: 0.1484878874769098}

## Sort the rank

In [18]:
ranked_sentence = sorted(((scores[i],file_names[i],s) for i,s in enumerate(full_sentences)), reverse=True)    
print("Indexes of top ranked_sentence order are ", ranked_sentence)
ranked_sentence

Indexes of top ranked_sentence order are  [(0.1484878874769098, '19-85-6.txt', ['Ada', 'ulama', 'yang', 'memahami', 'ayat', 'ini', 'dan', 'semacamnya', 'sebagai', 'berbicara', 'tentang', 'orang', 'kafir', 'sehingga', 'pembelaan', 'dan', 'syafaat', 'yang', 'dinafikan', 'adalah', 'yang', 'bersumber', 'dari', 'orang-orang', 'kafir', 'kepada', 'orang', 'kafir.', '']), (0.14436361369548853, '19-85-10.txt', ['"Dan', 'sembahan-sembahan', 'yang', 'mereka', 'sembah', 'selain', 'Allah', 'tidak', 'dapat', 'memberi', 'syafaat;', 'akan', 'tetapi', 'orang', 'yang', 'dapat', 'memberi', 'syafaat', 'ialah', 'yang', 'mengakui', 'yang', 'Haq,', 'yakni', 'keesaan', 'Allah', 'dan', 'mereka', 'meyakininya"', '(QS.', 'az-Zukhruf', '[43]:', '86).', 'Demikian', 'juga', 'firman-Nya:', 'orang', '"Tiadalah', 'berguna', 'syafaat', 'di', 'sisi', 'Allah', 'melainkan', 'bagi', 'orang', 'yang', 'telah', 'diizinkan-Nya', 'memeroleh', 'syafaat', 'itu', 'sehingga', 'apabila', 'telah', 'dihilangkan', 'ketakutan', 'dari', 

[(0.1484878874769098,
  '19-85-6.txt',
  ['Ada',
   'ulama',
   'yang',
   'memahami',
   'ayat',
   'ini',
   'dan',
   'semacamnya',
   'sebagai',
   'berbicara',
   'tentang',
   'orang',
   'kafir',
   'sehingga',
   'pembelaan',
   'dan',
   'syafaat',
   'yang',
   'dinafikan',
   'adalah',
   'yang',
   'bersumber',
   'dari',
   'orang-orang',
   'kafir',
   'kepada',
   'orang',
   'kafir.',
   '']),
 (0.14436361369548853,
  '19-85-10.txt',
  ['"Dan',
   'sembahan-sembahan',
   'yang',
   'mereka',
   'sembah',
   'selain',
   'Allah',
   'tidak',
   'dapat',
   'memberi',
   'syafaat;',
   'akan',
   'tetapi',
   'orang',
   'yang',
   'dapat',
   'memberi',
   'syafaat',
   'ialah',
   'yang',
   'mengakui',
   'yang',
   'Haq,',
   'yakni',
   'keesaan',
   'Allah',
   'dan',
   'mereka',
   'meyakininya"',
   '(QS.',
   'az-Zukhruf',
   '[43]:',
   '86).',
   'Demikian',
   'juga',
   'firman-Nya:',
   'orang',
   '"Tiadalah',
   'berguna',
   'syafaat',
   'di',
   'sisi'

In [19]:

#top_n = int(len(scores)/2)
top_n = 5
summarize_text = []
for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][2]))

In [20]:
summarize_text

['Ada ulama yang memahami ayat ini dan semacamnya sebagai berbicara tentang orang kafir sehingga pembelaan dan syafaat yang dinafikan adalah yang bersumber dari orang-orang kafir kepada orang kafir. ',
 '"Dan sembahan-sembahan yang mereka sembah selain Allah tidak dapat memberi syafaat; akan tetapi orang yang dapat memberi syafaat ialah yang mengakui yang Haq, yakni keesaan Allah dan mereka meyakininya" (QS. az-Zukhruf [43]: 86). Demikian juga firman-Nya: orang "Tiadalah berguna syafaat di sisi Allah melainkan bagi orang yang telah diizinkan-Nya memeroleh syafaat itu sehingga apabila telah dihilangkan ketakutan dari hati mereka, mereka berkata: \'Apakah yang telah difirmankan oleh Tuhan-mu?’ Mereka menjawab: Putusan yang benar\', dan Dia-lah Yang Mahatinggi lagi Mahabesar" (QS. Saba\' [34]: 23). Memang, syafaat ala dunia tidak akan terjadi di hari Kemudian, dan itulah yang dinafikan oleh sekian ayat. ',
 'Ada lagi yang berpendapat bahwa pembelaan dan pemberian syafaat yang dari siapa p

In [21]:
str_summarize = ''.join(summarize_text)
print("Hasil Rangkuman {} rank teratas : \n{}".format(top_n, str_summarize))

Hasil Rangkuman 5 rank teratas : 
Ada ulama yang memahami ayat ini dan semacamnya sebagai berbicara tentang orang kafir sehingga pembelaan dan syafaat yang dinafikan adalah yang bersumber dari orang-orang kafir kepada orang kafir. "Dan sembahan-sembahan yang mereka sembah selain Allah tidak dapat memberi syafaat; akan tetapi orang yang dapat memberi syafaat ialah yang mengakui yang Haq, yakni keesaan Allah dan mereka meyakininya" (QS. az-Zukhruf [43]: 86). Demikian juga firman-Nya: orang "Tiadalah berguna syafaat di sisi Allah melainkan bagi orang yang telah diizinkan-Nya memeroleh syafaat itu sehingga apabila telah dihilangkan ketakutan dari hati mereka, mereka berkata: 'Apakah yang telah difirmankan oleh Tuhan-mu?’ Mereka menjawab: Putusan yang benar', dan Dia-lah Yang Mahatinggi lagi Mahabesar" (QS. Saba' [34]: 23). Memang, syafaat ala dunia tidak akan terjadi di hari Kemudian, dan itulah yang dinafikan oleh sekian ayat. Ada lagi yang berpendapat bahwa pembelaan dan pemberian syafaa

In [20]:
## Integrate w